In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride,
                               padding=padding, dilation=dilation)
        self.batch_norm1 = nn.BatchNorm1d(out_channels)
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=stride,
                               padding=padding, dilation=dilation)
        self.batch_norm2 = nn.BatchNorm1d(out_channels)
        self.dropout2 = nn.Dropout(dropout)

        self.downsample = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None

        # Initializing weights
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = F.relu(self.batch_norm1(self.conv1(x)))
        out = self.dropout1(out)
        out = F.relu(self.batch_norm2(self.conv2(out)))
        out = self.dropout2(out)

        # Downsample (if necessary)
        res = x if self.downsample is None else self.downsample(x)

        # Ensure the sequence length matches
        if res.shape[2] != out.shape[2]:
            diff = res.shape[2] - out.shape[2]
            out = F.pad(out, (0, diff))

        return F.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i - 1]
            out_channels = num_channels[i]
            padding = (kernel_size - 1) * dilation_size

            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=padding, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


class TCNClassifier(nn.Module):
    def __init__(self, input_size, num_classes, num_channels, kernel_size=2, dropout=0.2):
        super(TCNClassifier, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout)
        self.linear = nn.Linear(num_channels[-1], num_classes)

    def forward(self, x):
        y = self.tcn(x)
        y = y[:, :, -1]  # Use the last output for classification
        y = self.linear(y)
        return F.softmax(y, dim=1) 


In [12]:
# Example data
batch_size = 10000
sequence_length = 30
num_features = 2  # Input size
num_classes = 10  # Number of classes

# Random data for demonstration
x = torch.randn(batch_size, num_features, sequence_length)
y = torch.randint(0, num_classes, (batch_size,))

# Define model, loss, and optimizer
model = TCNClassifier(input_size=num_features, num_classes=num_classes, num_channels=[25, 50, 75])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop (simplified)
model.train()
for epoch in range(10):  # Number of epochs
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 2.304046630859375
Epoch 2, Loss: 2.303402900695801
Epoch 3, Loss: 2.3032405376434326
Epoch 4, Loss: 2.3026256561279297
Epoch 5, Loss: 2.301750898361206
Epoch 6, Loss: 2.3014299869537354
Epoch 7, Loss: 2.300898313522339
Epoch 8, Loss: 2.3011722564697266
Epoch 9, Loss: 2.300260066986084
Epoch 10, Loss: 2.300034523010254


In [9]:
len(y)

10000

In [ ]:
# Example data
batch_size = 32
sequence_length = 100
num_features = 10  # Input size
num_classes = 5  # Number of classes

# Random data for demonstration
x = torch.randn(batch_size, num_features, sequence_length)
y = torch.randint(0, num_classes, (batch_size,))

# Define model, loss, and optimizer
model = TCNClassifier(input_size=num_features, num_classes=num_classes, num_channels=[25, 50, 75])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop (simplified)
model.train()
for epoch in range(10):  # Number of epochs
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
